# FedCAD: Centralized vs Federated (orchestration)

This notebook launches centralized and federated experiments via existing entry points:
- Centralized: `python train_central.py`
- Federated: `flwr run .`

Metrics tracked in W&B:
- Centralized: `train_loss`, `test_loss`, `test_acc` (already logged by `train_central.py`)
- Federated: depends on your Flower app (ensure it logs to the same W&B project/group)

Recommended seeds for a seminar: 3 (e.g., `[0, 1, 2]`). Use 5 if time permits for tighter confidence intervals.

In [32]:
import os, sys, subprocess, shutil, time
import torch

# W&B configuration
def load_wandb_config():
    """Load WandB credentials from config file (optional for reproducibility)"""
    config = {}
    config_path = './.wandb_config'
    try:
        with open(config_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    key, value = line.split('=', 1)
                    config[key] = value
        return config.get('WANDB_ENTITY'), config.get('WANDB_PROJECT')
    except FileNotFoundError:
        return None, None

entity, project_name = load_wandb_config()
WANDB_MODE = "online"               # set to "offline" or "disabled" if needed

# Experiment seeds (3 is seminar-friendly)
SEEDS = [0, 1, 2, 3, 4, 5]

# Centralized hyperparameters
EPOCHS_CENTRAL = 10
LR_CENTRAL = 1e-3
EVAL_INTERVAL = 1

# Device
USE_GPU_FLAG = torch.cuda.is_available()
print("Device:", "cuda" if USE_GPU_FLAG else "cpu")
print("Seeds:", SEEDS)

def run_cmd(cmd_list, extra_env=None, cwd=None):
    env = os.environ.copy()
    if extra_env:
        env.update({k: str(v) for k, v in extra_env.items()})
    print("\n>>>", " ".join(cmd_list))
    print("    in:", cwd or os.getcwd())
    print("    with env overrides:", extra_env or {})
    proc = subprocess.Popen(cmd_list, cwd=cwd, env=env)
    proc.wait()
    if proc.returncode != 0:
        raise RuntimeError(f"Command failed with exit code {proc.returncode}: {' '.join(cmd_list)}")

Device: cpu
Seeds: [0, 1, 2, 3, 4, 5]


## Centralized experiments

Runs `train_central.py` once per seed. The script already logs: `train_loss`, `test_loss`, `test_acc`.

Note: `train_central.py` does not expose a `--seed` flag. If you need strict reproducibility per seed, add seeding to the script; otherwise each run will use random initialization.

In [ ]:
for seed in SEEDS:
    # W&B grouping so runs are easy to compare
    env_overrides = {
        "WANDB_PROJECT": project_name,
        "WANDB_MODE": WANDB_MODE,
        "WANDB_RUN_GROUP": "centralized",
        "WANDB_TAGS": "centralized,notebook",
        "WANDB_ENTITY": entity,

        # Optional: many apps read SEED from env; train_central.py currently does not
        "SEED": seed,
    }

    cmd = [sys.executable, "train_central.py",
           "--epochs", str(EPOCHS_CENTRAL),
           "--lr", str(LR_CENTRAL),
           "--eval-interval", str(EVAL_INTERVAL),
           "--wandb-mode", WANDB_MODE]
    if USE_GPU_FLAG:
        cmd.append("--gpu")

    print(f"\n=== Centralized run, seed={seed} ===")
    run_cmd(cmd, extra_env=env_overrides, cwd=os.getcwd())
    time.sleep(1)  # small gap between runs

## Federated experiments (Flower Simulation Engine)

Runs `flwr run .` once per seed. Ensure your Flower app logs to W&B (same project) with metrics comparable to centralized (e.g., `global_train_loss`, `global_test_loss`, `global_test_acc`).

Tip: If your Flower app reads `WANDB_*` env vars and `SEED`, the grouping and seeds below will apply.

In [34]:
def run_flower(extra_env=None):
    # Prefer CLI if available; fallback to `python -m flwr`
    if shutil.which("flwr"):
        run_cmd(["flwr", "run", "."], extra_env=extra_env, cwd=os.getcwd())
    else:
        run_cmd([sys.executable, "-m", "flwr", "run", "."], extra_env=extra_env, cwd=os.getcwd())

entity, project_name = load_wandb_config()

for seed in SEEDS:
    env_overrides = {
        "WANDB_PROJECT": project_name,
        "WANDB_MODE": WANDB_MODE,
        "WANDB_RUN_GROUP": "federated",
        "WANDB_TAGS": "federated,notebook",
        "WANDB_ENTITY": entity,
        # Many FL apps use this to control randomness; ensure your app reads it
        "SEED": seed,
    }
    print(f"\n=== Federated run (Flower), seed={seed} ===")
    run_flower(extra_env=env_overrides)
    time.sleep(1)


=== Federated run (Flower), seed=0 ===

>>> flwr run .
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FedCAD', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'federated', 'WANDB_TAGS': 'federated,notebook', 'WANDB_ENTITY': 'louisewiljander-ludwig-maximilianuniversity-of-munich', 'SEED': 0}
Loading project configuration... 
Success


wandb: Currently logged in as: zoematr (louisewiljander-ludwig-maximilianuniversity-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/Users/zoe/Desktop/Fed-CAD/.venv/lib/python3.12/site-packages/wandb/analytics/sentry.py:279: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}
wandb: setting up run trxncvpa
/Users/zoe/Desktop/Fed-CAD/.venv/lib/python3.12/site-packages/wandb/analytics/sentry.py:279: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}
wandb: Tracking run with wandb version 0.23.0
wandb: Run data is saved locally in /Users/zoe/Desktop/Fed-CAD/wandb/run-20260223_203541-trxncvpa
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run federated_lr0.001_r10_le1
wandb: ⭐️ View project at https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD
wandb: 🚀 View run at ht


Final Results | test_loss=0.472 | test_acc=0.764
Metrics: prec=0.583, rec=0.764, f1=0.661, auc=0.752
Total training time: 0.18 minutes

Saving final model to disk...


wandb: updating run metadata
wandb: uploading config.yaml
wandb: uploading summary, console lines 0-134
wandb: 
wandb: Run summary:
wandb:           final_auc_roc 0.75183
wandb:                final_f1 0.66129
wandb:         final_precision 0.58314
wandb:            final_recall 0.76364
wandb:          final_test_acc 0.76364
wandb:         final_test_loss 0.47218
wandb:      total_local_epochs 10
wandb:            total_rounds 10
wandb: total_training_time_min 0.18107
wandb: 
wandb: 🚀 View run federated_lr0.001_r10_le1 at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD/runs/trxncvpa
wandb: ⭐️ View project at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20260223_203541-trxncvpa/logs



=== Federated run (Flower), seed=1 ===

>>> flwr run .
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FedCAD', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'federated', 'WANDB_TAGS': 'federated,notebook', 'WANDB_ENTITY': 'louisewiljander-ludwig-maximilianuniversity-of-munich', 'SEED': 1}
Loading project configuration... 
Success


...


Final Results | test_loss=0.485 | test_acc=0.764
Metrics: prec=0.583, rec=0.764, f1=0.661, auc=0.767
Total training time: 0.22 minutes

Saving final model to disk...


wandb: uploading config.yaml
wandb: 
wandb: Run summary:
wandb:           final_auc_roc 0.7674
wandb:                final_f1 0.66129
wandb:         final_precision 0.58314
wandb:            final_recall 0.76364
wandb:          final_test_acc 0.76364
wandb:         final_test_loss 0.4846
wandb:      total_local_epochs 10
wandb:            total_rounds 10
wandb: total_training_time_min 0.22375
wandb: 
wandb: 🚀 View run federated_lr0.001_r10_le1 at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD/runs/b2yslcw5
wandb: ⭐️ View project at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20260223_203606-b2yslcw5/logs



=== Federated run (Flower), seed=2 ===

>>> flwr run .
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FedCAD', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'federated', 'WANDB_TAGS': 'federated,notebook', 'WANDB_ENTITY': 'louisewiljander-ludwig-maximilianuniversity-of-munich', 'SEED': 2}
Loading project configuration... 
Success


wandb: Currently logged in as: zoematr (louisewiljander-ludwig-maximilianuniversity-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/Users/zoe/Desktop/Fed-CAD/.venv/lib/python3.12/site-packages/wandb/analytics/sentry.py:279: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}
wandb: setting up run nurji9b9
/Users/zoe/Desktop/Fed-CAD/.venv/lib/python3.12/site-packages/wandb/analytics/sentry.py:279: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}
wandb: Tracking run with wandb version 0.23.0
wandb: Run data is saved locally in /Users/zoe/Desktop/Fed-CAD/wandb/run-20260223_203632-nurji9b9
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run federated_lr0.001_r10_le1
wandb: ⭐️ View project at https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD
wandb: 🚀 View run at ht

KeyboardInterrupt: 